In [1]:
import gym
import pybullet_envs
import pybullet

import ray
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
from ray.tune.registry import register_env
import time
from scipy.spatial.transform import Rotation as R
import pandas as pd
import numpy as np

from low_level_env import LowLevelHumanoidEnv
from hier_env import HierarchicalHumanoidEnv

from pymo.parsers import BVHParser
from pymo.preprocessing import *

from humanoid import CustomHumanoidRobot

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def make_env(env_config):
    import pybullet_envs
    return LowLevelHumanoidEnv(reference_name="motion09_03")


ENV = 'HumanoidBulletEnv-Low-v0'
register_env(ENV, make_env)
TARGET_REWARD = 2000

In [3]:
parser = BVHParser()
parsed_data = parser.parse('Dataset/CMU_Mocap_BVH/09/09_03.bvh')
mp = MocapParameterizer('position')
bvh_pos = mp.fit_transform([parsed_data])[0].values

In [4]:
for c in bvh_pos.columns:
    print(c)

Hips_Xposition
Hips_Yposition
Hips_Zposition
LowerBack_Xposition
LowerBack_Yposition
LowerBack_Zposition
Spine_Xposition
Spine_Yposition
Spine_Zposition
Spine1_Xposition
Spine1_Yposition
Spine1_Zposition
RightShoulder_Xposition
RightShoulder_Yposition
RightShoulder_Zposition
RightArm_Xposition
RightArm_Yposition
RightArm_Zposition
RightForeArm_Xposition
RightForeArm_Yposition
RightForeArm_Zposition
RightHand_Xposition
RightHand_Yposition
RightHand_Zposition
RThumb_Xposition
RThumb_Yposition
RThumb_Zposition
RThumb_Nub_Xposition
RThumb_Nub_Yposition
RThumb_Nub_Zposition
RightFingerBase_Xposition
RightFingerBase_Yposition
RightFingerBase_Zposition
RightHandIndex1_Xposition
RightHandIndex1_Yposition
RightHandIndex1_Zposition
RightHandIndex1_Nub_Xposition
RightHandIndex1_Nub_Yposition
RightHandIndex1_Nub_Zposition
LeftShoulder_Xposition
LeftShoulder_Yposition
LeftShoulder_Zposition
LeftArm_Xposition
LeftArm_Yposition
LeftArm_Zposition
LeftForeArm_Xposition
LeftForeArm_Yposition
LeftForeArm

In [5]:
def getNormalJointPos(df, joint, frame):
    x = df.iloc[frame][joint + '_Xposition']
    y = df.iloc[frame][joint + '_Yposition']
    z = df.iloc[frame][joint + '_Zposition']
    return np.array([x, y, z])

def _getJointPos(df, joint, frame):
    x = df.iloc[frame][joint + '_Xposition']
    z = df.iloc[frame][joint + '_Zposition']
    y = df.iloc[frame][joint + '_Yposition']
    return np.array([z, x, y])/17

def getJointPos(df, joint, frame):
    xStart, yStart, zStart = _getJointPos(df, 'Hips', 1)
    return _getJointPos(df, joint, frame) - np.array([xStart, yStart, 0.01])

def getVectorFrom2Joint(df, j1, j2, frame):
    return getJointPos(df, j2, frame) - getJointPos(df, j1, frame)

In [6]:
endpoint = ['LeftLeg', 'LeftFoot', 'RightLeg', 'RightFoot', 'RightShoulder', 'RightArm', 'RightForeArm', 'RightHand', 'LeftShoulder', 'LeftArm', 'LeftForeArm', 'LeftHand', 'RightToeBase', 'LeftToeBase']
normalized_data = np.zeros((1, 3 * len(endpoint)))
for i in range(1, len(bvh_pos)):
    basePos = _getJointPos(bvh_pos, 'Hips', i)
    tmp = []
    for ep in endpoint:
        epPos = _getJointPos(bvh_pos, ep, i)
        tmp = np.hstack((tmp, epPos - basePos))
    normalized_data = np.vstack((normalized_data, tmp))
normalized_data = normalized_data[1:]

In [7]:
norm_df = pd.DataFrame(normalized_data, columns=['{}_{}position'.format(ep, axis) for ep in endpoint for axis in ['X', 'Y', 'Z']])

In [8]:
norm_df.head()

,LeftLeg_Xposition,LeftLeg_Yposition,LeftLeg_Zposition,LeftFoot_Xposition,LeftFoot_Yposition,LeftFoot_Zposition,RightLeg_Xposition,RightLeg_Yposition,RightLeg_Zposition,RightFoot_Xposition,...,LeftForeArm_Zposition,LeftHand_Xposition,LeftHand_Yposition,LeftHand_Zposition,RightToeBase_Xposition,RightToeBase_Yposition,RightToeBase_Zposition,LeftToeBase_Xposition,LeftToeBase_Yposition,LeftToeBase_Zposition
0,0.132066,0.058209,-0.539438,-0.111781,0.021125,-0.920638,0.234594,-0.114282,-0.500467,-0.218909,...,-0.030834,0.223964,0.163998,0.031965,-0.248700,-0.088121,-0.675125,0.005024,0.015073,-0.973826
1,0.114431,0.056799,-0.542626,-0.138951,0.019221,-0.917507,0.252191,-0.114436,-0.491098,-0.198343,...,-0.032305,0.235206,0.159763,0.038723,-0.224966,-0.086353,-0.690158,-0.023658,0.013861,-0.973968
2,0.096141,0.056069,-0.544677,-0.163915,0.017527,-0.914860,0.268597,-0.114195,-0.481995,-0.177090,...,-0.032734,0.244698,0.154913,0.044777,-0.196305,-0.085118,-0.708934,-0.050540,0.012435,-0.975102
3,0.076973,0.056302,-0.545964,-0.189943,0.017096,-0.911162,0.282271,-0.112644,-0.474022,-0.157003,...,-0.032444,0.252601,0.150792,0.050529,-0.165442,-0.083605,-0.728445,-0.079024,0.012018,-0.975817
4,0.056707,0.056930,-0.546975,-0.216218,0.018315,-0.907768,0.293448,-0.110305,-0.466818,-0.137418,...,-0.031731,0.259607,0.147845,0.056203,-0.132572,-0.082255,-0.748524,-0.108247,0.012868,-0.977207


In [9]:
# norm_df.to_csv('Joints CSV/walk8_1JointVecFromHip.csv', index=False)

In [10]:
getNormalJointPos(norm_df, 'LeftLeg', 0)

array([ 0.13206635,  0.0582094 , -0.53943767])

In [11]:
getVectorFrom2Joint(bvh_pos, 'Hips', 'LeftLeg', 0)

array([ 0.03751941,  0.08479418, -0.55291021])

In [12]:
def quatFromJoints(joints, idx, joint_name):
    x, y, z, w = joints.iloc[idx][joint_name + '.X'], joints.iloc[idx][joint_name +
                                                   '.Y'], joints.iloc[idx][joint_name + '.Z'], joints.iloc[idx][joint_name + '.W']
    return R.from_quat([x, y, z, w]).as_quat().tolist()

def eulerFromJoints(joints, idx, joint_name, order='xyz'):
    # return pybullet.getEulerFromQuaternion(quatFromJoints(joints, idx, joint_name))
    x, y, z = [np.deg2rad(joints.iloc[idx][joint_name+o]) for o in ['.X', '.Y', '.Z']]
    # return R.from_euler('zyx', [z, y, x]).as_euler(order)
    return [x, y, z]

In [13]:
def rotFrom2Vec(v1, v2):
    rotAxis= np.cross(v1, v2)
    rotAxis = rotAxis / np.linalg.norm(rotAxis)
    rotAngle = v1.dot(v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    rotAngle = np.arccos(rotAngle)
    if(np.all(rotAxis == 0)):
        rotAxis = np.array([1, 0, 0])
        if((v1 == v2).all()):
            rotAngle = 0
        else:
            rotAngle = np.pi
    return R.from_rotvec(rotAxis * rotAngle)

In [14]:
v1 = np.array([3, 3, 21])
v2 = np.array([2, 1, 5])
r = rotFrom2Vec(v1, v2)
print(r.as_euler('xyz', degrees=True))
print(r.apply(v1) / np.linalg.norm(v1) * np.linalg.norm(v2))

[-3.18715833 13.26330681 -1.85654086]
[2. 1. 5.]


In [15]:
def drawAxis():
    xId = pybullet.addUserDebugLine([0, 0, 0], [10, 0, 0], lineColorRGB=[1, 0, 0], lineWidth=2)
    yId = pybullet.addUserDebugLine([0, 0, 0], [0, 5, 0], lineColorRGB=[0, 1, 0], lineWidth=2)
    zId = pybullet.addUserDebugLine([0, 0, 0], [0, 0, 15], lineColorRGB=[0, 0, 1], lineWidth=2)
    return xId, yId, zId

def drawLine2Joint(df, j1, j2, frame, color):
    return pybullet.addUserDebugLine(getJointPos(df, j1, frame), getJointPos(df, j2, frame), lineColorRGB=color, lineWidth=5)

def getVector2Parts(parts, p1, p2):
    return parts[p2].get_position() - parts[p1].get_position()

def getVector2Joint(df, j1, j2, frame):
    return getJointPos(df, j2, frame) - getJointPos(df, j1, frame)

def angle2Vec(v1, v2):
    return np.arccos(v1.dot(v2)/(np.linalg.norm(v1) * np.linalg.norm(v2)))

def drawLine2Parts(env, p1, p2, color):
    delta = np.array([1, 0, 0])
    return pybullet.addUserDebugLine(env.parts[p1].get_position() + delta, env.parts[p1].get_position() + delta, lineColorRGB=color, lineWidth=5)

def drawLine(c1, c2, color):
    return pybullet.addUserDebugLine(c1, c2, lineColorRGB=color, lineWidth=5)

def draw2(p, color):
    return drawLine([-3, 0, 0], env.flat_env.parts[p].get_position(), color)

In [26]:
env.close()

In [16]:
env = LowLevelHumanoidEnv(reference_name="motion09_03", customRobot=CustomHumanoidRobot())

In [17]:
env.render()
observation = env.resetFromFrame(startFrame=0, resetYaw=0, startFromRef=True)
pybullet.removeAllUserDebugItems()
drawAxis()

(0, 1, 2)

In [18]:
observation

array([ 0.37      ,  0.12967981,  0.99155593,  1.87121713,  0.01112592,
        0.45616901,  0.        ,  0.        ,  0.        ,  0.        ,
        0.42857143,  0.        ,  0.        ,  0.        ,  0.370334  ,
        0.        ,  0.42322886,  0.        ,  0.27030659,  0.        ,
        0.23502456,  0.        , -0.13674481,  0.        , -0.46700674,
        0.        ,  0.76509839,  0.        ,  0.29358155,  0.        ,
        0.47564051,  0.        ,  0.45783779,  0.        ,  0.5352118 ,
        0.        , -0.30779222,  0.        ,  0.68367165,  0.        ,
        0.07500515,  0.        , -0.06618293,  0.        ,  0.37244859,
        0.        ,  0.70054132,  0.        ,  0.05114443,  0.        ,
       -0.36034118,  1.41404871,  0.25343593, -0.26496762,  0.19300883,
       -2.19495334,  0.41009348, -0.02384195,  0.47840375,  1.47231628,
        0.71364225,  0.26732834,  0.54626803,  2.624408  ,  0.29926521,
        0.26844835,  0.33556914,  0.52268237,  0.70786169,  0.90

In [25]:
for i in range(100):
    observation, reward, f_done, info = env.step(np.array([
        0, 0, 0,
        0, 0, 0, 0,
        0, 0, 0, 0,
        0, 0, 0,
        0, 0, 0,
        -1, 1, -1, 1
    ]))
    time.sleep(0.0165)

In [19]:
basePos = env.flat_env.parts['lwaist'].get_position()
pybullet.removeAllUserDebugItems()

target_pos = np.array([1, 0, 0])
drawLine([0, 0, 0], target_pos, [0, 0, 0])
r = R.from_euler('z', np.arctan2(target_pos[1], target_pos[0]))
env.flat_env.parts['torso'].reset_orientation(r.as_quat())
env.flat_env.parts['torso'].reset_position([0, 1, 1.15])

# vll = basePos + r.apply(getNormalJointPos(norm_df, 'LeftLeg', 10))
# vlf = basePos + r.apply(getNormalJointPos(norm_df, 'LeftFoot', 10))
# drawLine(basePos, vll, [1, 0, 0])
# drawLine(basePos, vlf, [0, 1, 0])

# vrl = basePos + r.apply(getNormalJointPos(norm_df, 'RightLeg', 10))
# vrf = basePos + r.apply(getNormalJointPos(norm_df, 'RightFoot', 10))
# drawLine(basePos, vrl, [0, 0, 1])
# drawLine(basePos, vrf, [1, 1, 1])

In [19]:
np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])[2:2+6]

array([2, 3, 4, 5, 6, 7])

In [22]:
v1 = env.flat_env.parts['left_foot'].get_position() 
v2 = basePos + getNormalJointPos(norm_df, 'LeftFoot', 10)

In [23]:
drawLine([0, 0, 0], v1, [0, 0, 1])
drawLine([0, 0, 0], v2, [0, 1, 0])
drawLine(v1, v2, [1, 1, 1])

3

In [23]:
np.linalg.norm(v2 - v1)

0.19855298780610323

In [24]:
drawLine(v1, v2, [1, 1, 1])

8

In [26]:
env.resetFromFrame(50)

array([ 3.4999999e-01, -1.3717048e-05,  1.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -0.0000000e+00,
        7.1336195e-02,  0.0000000e+00,  5.1822627e-01,  0.0000000e+00,
        1.2700889e-02,  0.0000000e+00,  1.1723903e+00,  0.0000000e+00,
        4.3820977e-01,  0.0000000e+00,  4.4546205e-01,  0.0000000e+00,
        5.5154860e-01,  0.0000000e+00,  9.6816510e-02,  0.0000000e+00,
        3.9674813e-01,  0.0000000e+00,  4.4077528e-01,  0.0000000e+00,
        1.2652647e-01,  0.0000000e+00,  1.2998933e-01,  0.0000000e+00,
        1.6158226e-01,  0.0000000e+00,  2.4842025e-01,  0.0000000e+00,
       -1.5762837e-01,  0.0000000e+00, -2.1543621e-01,  0.0000000e+00,
        2.8703377e-01,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
      dtype=float32)

In [68]:
env.close()

In [30]:
# pybullet.removeAllUserDebugItems()
# idx = 100
# v1 = getVector2Parts(env.flat_env.parts, 'link0_15', 'link0_19')
# v2 = getVector2Joint(bvh_pos, 'LeftUpLeg', 'LeftLeg', idx)

# r = rotFrom2Vec(v1, v2)
# rE = r.as_euler('xyz')
# print(rE)

# jx = env.flat_env.jdict['left_hip_x'].get_position()
# jy = env.flat_env.jdict['left_hip_y'].get_position()
# jz = env.flat_env.jdict['left_hip_z'].get_position()

# env.flat_env.jdict['left_hip_x'].set_state(jx - rE[0], 0)
# env.flat_env.jdict['left_hip_y'].set_state(jy + rE[1], 0)
# env.flat_env.jdict['left_hip_z'].set_state(jz - rE[2], 0)

# v11 = env.flat_env.parts['link0_15'].get_position()
# v12 = env.flat_env.parts['link0_19'].get_position()

# v21 = getJointPos(bvh_pos, 'LeftUpLeg', idx)
# v22 = getJointPos(bvh_pos, 'LeftLeg', idx)

# drawLine([-3, 0, 0], v11, [1, 1, 1])
# drawLine([-3, 0, 0], v12, [0, 0, 1])
# drawLine(v21, v21 + (v12- v11), [1, 1, 1])

# drawLine([-3, 0, 0], v21, [1, 0, 0])
# drawLine([-3, 0, 0], v22, [0, 1, 0])
# drawLine(v21, v22, [1, 1, 0])

In [20]:
env.flat_env.jdict

{'abdomen_z': <pybullet_envs.robot_bases.Joint at 0x7fe8a47007c0>,
 'abdomen_y': <pybullet_envs.robot_bases.Joint at 0x7fe9cc1a64f0>,
 'abdomen_x': <pybullet_envs.robot_bases.Joint at 0x7fe8a4700ee0>,
 'right_hip_x': <pybullet_envs.robot_bases.Joint at 0x7fe8a49cd100>,
 'right_hip_z': <pybullet_envs.robot_bases.Joint at 0x7fe8a49cd4c0>,
 'right_hip_y': <pybullet_envs.robot_bases.Joint at 0x7fe8a49cd610>,
 'right_knee': <pybullet_envs.robot_bases.Joint at 0x7fe8a49cd7f0>,
 'right_ankle_y': <pybullet_envs.robot_bases.Joint at 0x7fe8a49cd4f0>,
 'right_ankle_x': <pybullet_envs.robot_bases.Joint at 0x7fe8a49cd640>,
 'left_hip_x': <pybullet_envs.robot_bases.Joint at 0x7fe8a49cd550>,
 'left_hip_z': <pybullet_envs.robot_bases.Joint at 0x7fe8a49cdd90>,
 'left_hip_y': <pybullet_envs.robot_bases.Joint at 0x7fe8a49cdca0>,
 'left_knee': <pybullet_envs.robot_bases.Joint at 0x7fe8a49cd8e0>,
 'left_ankle_y': <pybullet_envs.robot_bases.Joint at 0x7fe8a49cdac0>,
 'left_ankle_x': <pybullet_envs.robot_bas

In [84]:
np.linalg.norm([.18, .18, .18])

0.3117691453623979

In [21]:
def getRot(parts, l1, l2, df, j1, j2, frame):
    v1 = getVector2Parts(parts, l1, l2)
    v2 = getVector2Joint(df, j1, j2, frame)
    return rotFrom2Vec(v1, v2).as_euler('xyz')

def moveJoint(l1, l2, j1, j2, frame, part_name, weight, index):
    rot = getRot(env.flat_env.parts, l1, l2, bvh_pos, j1, j2, frame)
    cur_pos = [env.flat_env.jdict[p].get_position() for p in part_name]
    
    pos = []
    for i, idx in enumerate(index):
        joint_pos = cur_pos[i] + rot[idx] * weight[i]
        env.flat_env.jdict[part_name[i]].set_state(joint_pos, 0)
        pos.append(joint_pos)
    return pos

# Link 7,8, 9 = right hip x, y, z
# Link 11 = right knee
# Link 13, 14 = right ankle x, y

# Link 16, 17, 18 = left hip x, y, z
# Link 20 = left knee
# Link 22, 23 = left ankle x, y

# Link 25, 26 = right shoulder y, x
# Link 28 = right elbow

# Link 31, 32 = left shoulder y, x
# Link 34 = left elbow
def setJoint(frame):
    rightHipX, rightHipY, rightHipZ = moveJoint('link0_7', 'link0_11', 'RightUpLeg', 'RightLeg', frame, ['right_hip_x', 'right_hip_y', 'right_hip_z'], [1, 1, 1], [0, 1, 2])
    rightKnee = moveJoint('link0_11', 'right_foot', 'RightLeg', 'RightFoot', frame, ['right_knee'], [-1], [1])[0]

    leftHipX, leftHipY,leftHipZ = moveJoint('link0_16', 'link0_20', 'LeftUpLeg', 'LeftLeg', frame, ['left_hip_x', 'left_hip_y', 'left_hip_z'], [-1, 1, -1], [0, 1, 2])
    leftKnee = moveJoint('link0_20', 'left_foot', 'LeftLeg', 'LeftFoot', frame, ['left_knee'], [-1], [1])[0]

    rightShoulderX, rightShoulderY = moveJoint('link0_25', 'link0_28', 'RightArm', 'RightForeArm', frame, ['right_shoulder2', 'right_shoulder1'], [1, 1], [0, 1])
    rightElbow = moveJoint('link0_28', 'right_hand', 'RightForeArm', 'RightHand', frame, ['right_elbow'], [1], [1])[0]

    leftShoulderX, leftShoulderY = moveJoint('link0_31', 'link0_34', 'LeftArm', 'LeftForeArm', frame, ['left_shoulder2', 'left_shoulder1'], [-1, 1], [0, 1])
    leftElbow = moveJoint('link0_34', 'left_hand', 'LeftForeArm', 'LeftHand', frame, ['left_elbow'], [1], [1])[0]

    return [rightHipX, rightHipY, rightHipZ, rightKnee, leftHipX, leftHipY, leftHipZ, leftKnee] + [rightShoulderX, rightShoulderY, rightElbow, leftShoulderX, leftShoulderY, leftElbow]

In [26]:
drawLine2Joint(bvh_pos, 'LowerBack', 'LeftUpLeg', i, [1, 0, 0])
drawLine2Joint(bvh_pos, 'LeftUpLeg', 'LeftLeg', i, [1, 0, 0])
drawLine2Joint(bvh_pos, 'LeftLeg', 'LeftFoot', i, [1, 0, 0])
drawLine2Joint(bvh_pos, 'LeftFoot', 'LeftToeBase', i, [0, 0, 0])

drawLine2Joint(bvh_pos, 'LowerBack', 'RightUpLeg', i, [0, 1, 0])
drawLine2Joint(bvh_pos, 'RightUpLeg', 'RightLeg', i, [0, 1, 0])
drawLine2Joint(bvh_pos, 'RightLeg', 'RightFoot', i, [0, 1, 0])
drawLine2Joint(bvh_pos, 'RightFoot', 'RightToeBase', i, [0, 0, 0])

drawLine2Joint(bvh_pos, 'Hips', 'RightShoulder', i, [1, 1,1 ])
drawLine2Joint(bvh_pos, 'RightShoulder', 'RightArm', i, [1, 1,1 ])
drawLine2Joint(bvh_pos, 'RightArm', 'RightForeArm', i, [1, 1,1 ])
drawLine2Joint(bvh_pos, 'RightForeArm', 'RightHand', i, [1, 1,1 ])

drawLine2Joint(bvh_pos, 'Hips', 'LeftShoulder', i, [0, 0, 0])
drawLine2Joint(bvh_pos, 'LeftShoulder', 'LeftArm', i, [0, 0, 0])
drawLine2Joint(bvh_pos, 'LeftArm', 'LeftForeArm', i, [0, 0, 0])
drawLine2Joint(bvh_pos, 'LeftForeArm', 'LeftHand', i, [0, 0, 0])

18

In [58]:
observation = env.resetFromFrame(startFrame=0, resetYaw=0, startFromRef=False)
pybullet.removeAllUserDebugItems()
drawAxis()
target_pos = np.array([1, 0, 0])
r = R.from_euler('z', np.arctan2(target_pos[1], target_pos[0]))
env.flat_env.parts['torso'].reset_orientation(r.as_quat())
env.flat_env.parts['torso'].reset_position([0, 1, 1.15])

In [28]:
env.flat_env.jdict['right_shoulder_x'].reset_position(np.deg2rad(0), 0)
env.flat_env.jdict['right_shoulder_y'].reset_position(np.deg2rad(0), 0)
env.flat_env.jdict['right_elbow'].reset_position(np.deg2rad(0), 0)

env.flat_env.jdict['left_shoulder_x'].reset_position(np.deg2rad(0), 0)
env.flat_env.jdict['left_shoulder_y'].reset_position(np.deg2rad(0), 0)
env.flat_env.jdict['left_elbow'].reset_position(np.deg2rad(0), 0)

In [67]:
env.flat_env.jdict['left_hip_x'].reset_position(np.deg2rad(0), 0)
env.flat_env.jdict['left_hip_y'].reset_position(np.deg2rad(0), 0)
env.flat_env.jdict['left_hip_z'].reset_position(np.deg2rad(0), 0)

env.flat_env.jdict['left_knee'].reset_position(np.deg2rad(0), 0)

env.flat_env.jdict['left_ankle_x'].reset_position(np.deg2rad(0), 0)
env.flat_env.jdict['left_ankle_y'].reset_position(np.deg2rad(-20), 0)

env.flat_env.jdict['right_hip_x'].reset_position(np.deg2rad(0), 0)
env.flat_env.jdict['right_hip_y'].reset_position(np.deg2rad(0), 0)
env.flat_env.jdict['right_hip_z'].reset_position(np.deg2rad(0), 0)

env.flat_env.jdict['right_knee'].reset_position(np.deg2rad(0), 0)

env.flat_env.jdict['right_ankle_x'].reset_position(np.deg2rad(-10), 0)
env.flat_env.jdict['right_ankle_y'].reset_position(np.deg2rad(-20), 0)

In [162]:
bvh_pos['RightUpLeg_Xposition']

0 days 00:00:00             -1.004860
0 days 00:00:00.008333300   -0.968415
0 days 00:00:00.016666600   -0.961939
0 days 00:00:00.024999900   -0.944482
0 days 00:00:00.033333200   -0.932186
                               ...   
0 days 00:00:01.033329200   -1.687926
0 days 00:00:01.041662500   -1.729985
0 days 00:00:01.049995800   -1.882531
0 days 00:00:01.058329100   -1.776120
0 days 00:00:01.066662400   -1.853478
Name: RightUpLeg_Xposition, Length: 129, dtype: float64

In [41]:
env.close()

In [18]:
env = LowLevelHumanoidEnv(reference_name="motion09_03", customRobot=CustomHumanoidRobot())
env.render()
observation = env.resetFromFrame(startFromRef=False)
target_pos = np.array([1, 0, 0])
r = R.from_euler('z', np.arctan2(target_pos[1], target_pos[0]))
env.flat_env.parts['torso'].reset_orientation(r.as_quat())
pybullet.removeAllUserDebugItems()
drawAxis()

joint_data = []
for i in range(1, 91):
    joint_data.append(setJoint(i))
    time.sleep(1.0/20)
# env.close()

In [143]:
joint_df = pd.DataFrame(joint_data, columns=['rightHipX', 'rightHipY', 'rightHipZ', 'rightKnee', 'leftHipX', 'leftHipY', 'leftHipZ', 'leftKnee', 'rightShoulderX', 'rightShoulderY', 'rightElbow', 'leftShoulderX', 'leftShoulderY', 'leftElbow'])

In [144]:
joint_df

,rightHipX,rightHipY,rightHipZ,rightKnee,leftHipX,leftHipY,leftHipZ,leftKnee,rightShoulderX,rightShoulderY,rightElbow,leftShoulderX,leftShoulderY,leftElbow
0,0.046257,-0.375044,0.077332,-0.817971,-0.030147,-0.380883,-0.018178,-0.778798,-0.214147,0.700747,-0.483390,-0.248510,0.685836,-0.499785
1,0.044828,-0.377821,0.076485,-0.840261,-0.058769,-0.376837,-0.031689,-0.778909,-0.195634,0.720963,-0.499220,-0.196411,0.717440,-0.527913
2,0.047112,-0.382826,0.077269,-0.849665,-0.059793,-0.381140,-0.032180,-0.784659,-0.189362,0.735669,-0.496448,-0.182863,0.729717,-0.525252
3,0.050092,-0.387496,0.078408,-0.857635,-0.061452,-0.387435,-0.032977,-0.792996,-0.187026,0.750574,-0.493342,-0.181321,0.742600,-0.522706
4,0.052537,-0.392605,0.079259,-0.865131,-0.062973,-0.393680,-0.033709,-0.800993,-0.185241,0.762538,-0.487135,-0.182781,0.754059,-0.518136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,0.016783,-0.630507,-0.017720,-1.355895,-0.088307,-0.604684,-0.108732,-1.292277,-0.152780,0.143170,-0.375359,-0.094878,0.239743,-0.352637
216,0.016620,-0.615055,-0.018391,-1.333185,-0.084677,-0.589146,-0.106871,-1.268116,-0.152682,0.142000,-0.374653,-0.092330,0.237688,-0.349631
217,0.016857,-0.601140,-0.018664,-1.310431,-0.082524,-0.575369,-0.106055,-1.244845,-0.153079,0.140854,-0.372880,-0.090195,0.235553,-0.346116
218,0.017920,-0.585286,-0.018328,-1.285548,-0.081100,-0.560575,-0.105853,-1.221340,-0.153298,0.140149,-0.370785,-0.088970,0.233551,-0.341908


In [117]:
joint_df.to_csv('Joints CSV/walk8_1JointPos.csv', index=False)

In [31]:
env.flat_env.parts

{'link0_2': <pybullet_envs.robot_bases.BodyPart at 0x7fe9cc1a6550>,
 'torso': <pybullet_envs.robot_bases.BodyPart at 0x7fe9cc1a6220>,
 'link0_3': <pybullet_envs.robot_bases.BodyPart at 0x7fe9cd24d550>,
 'lwaist': <pybullet_envs.robot_bases.BodyPart at 0x7fe8a4700fd0>,
 'link0_5': <pybullet_envs.robot_bases.BodyPart at 0x7fe8a4700f10>,
 'pelvis': <pybullet_envs.robot_bases.BodyPart at 0x7fe8a4700430>,
 'link0_7': <pybullet_envs.robot_bases.BodyPart at 0x7fe8a49cd730>,
 'link0_8': <pybullet_envs.robot_bases.BodyPart at 0x7fe8a4700940>,
 'link0_9': <pybullet_envs.robot_bases.BodyPart at 0x7fe8a49cd520>,
 'right_thigh': <pybullet_envs.robot_bases.BodyPart at 0x7fe8a49cd280>,
 'link0_11': <pybullet_envs.robot_bases.BodyPart at 0x7fe8a49cd190>,
 'right_shin': <pybullet_envs.robot_bases.BodyPart at 0x7fe8a49cd1f0>,
 'link0_13': <pybullet_envs.robot_bases.BodyPart at 0x7fe8a49cd0a0>,
 'link0_14': <pybullet_envs.robot_bases.BodyPart at 0x7fe8a49cd790>,
 'right_foot': <pybullet_envs.robot_bases.

In [40]:
pybullet.removeAllUserDebugItems()
# draw2('link0_7', [1, 0, 0]) # Kanan hip
# draw2('link0_11', [1, 0, 0]) # Kanan lutut
# draw2('right_foot', [1, 0, 0]) # Kanan telapak
# draw2('link0_14', [0, 1, 0]) # Kiri atas
# draw2('link0_18', [0, 1, 0]) # Kiri lutut
# draw2('left_foot', [0, 1, 0]) # Kiri telapak

# draw2('link0_13', [1, 0, 0]) # Kanan ankle
# draw2('link0_25', [0, 1, 0]) # Kanan ankle

# draw2('left_hand', [1, 1, 1]) # Telapak tangan kiri
# draw2('link0_30', [1, 1, 1]) # Siku Kiri
# draw2('link0_27', [1, 1, 1]) # Pundak kiri
# draw2('right_hand', [0, 0, 0]) # Telapak tangan kanan
# draw2('link0_24', [0, 0, 0]) # Siku kanan
# draw2('link0_21', [0, 0, 0]) # Pundak kanan

# Link 7,8, 9 = right hip x, y, z
# Link 11 = right knee
# Link 13, 14 = right ankle x, y

# Link 16, 17, 18 = left hip x, y, z
# Link 20 = left knee
# Link 22, 23 = left ankle x, y

# Link 25, 26 = right shoulder y, x
# Link 28 = right elbow

# Link 31, 32 = left shoulder y, x
# Link 34 = left elbow
draw2('link0_14', [1, 1, 1])

0

In [138]:
print(moveJoint('link0_21', 'link0_24', 'RightArm', 'RightForeArm', 150, ['right_shoulder2', 'right_shoulder1'], [1, 1], [0, 1]))

print(moveJoint('link0_24', 'right_hand', 'RightForeArm', 'RightHand', 1, ['right_elbow'], [1], [1])[0])

print(moveJoint('link0_27', 'link0_30', 'LeftArm', 'LeftForeArm', 150, ['left_shoulder2', 'left_shoulder1'], [-1, 1], [0, 1]))

print(moveJoint('link0_30', 'left_hand', 'LeftForeArm', 'LeftHand', 1, ['left_elbow'], [1], [1])[0])

[-0.18944999798197992, 1.0394105764832144]
-1.41042067781802
[-0.14738645053161248, 1.0931221036923697]
-1.3549473071365925


In [99]:
env.flat_env.jdict['right_shoulder1'].reset_position(np.deg2rad(0), 0)
env.flat_env.jdict['right_shoulder2'].reset_position(np.deg2rad(0), 0)
env.flat_env.jdict['right_elbow'].reset_position(np.deg2rad(0), 0)

env.flat_env.jdict['left_shoulder1'].reset_position(np.deg2rad(0), 0)
env.flat_env.jdict['left_shoulder2'].reset_position(np.deg2rad(0), 0)
env.flat_env.jdict['left_elbow'].reset_position(np.deg2rad(0), 0)

In [62]:
pybullet.removeAllUserDebugItems()

In [74]:
env.flat_env.parts['torso'].reset_position([5, 5, 0])

In [59]:
drawLine2Parts(env.flat_env, 'pelvis', 'left_thigh', [1, 1, 1])
drawLine2Parts(env.flat_env, 'left_thigh', 'left_shin', [1, 1, 1])
drawLine2Parts(env.flat_env, 'left_shin', 'left_foot', [1, 1, 1])

drawLine2Parts(env.flat_env, 'pelvis', 'right_thigh', [0, 0, 1])
drawLine2Parts(env.flat_env, 'right_thigh', 'right_shin', [0, 0, 1])
drawLine2Parts(env.flat_env, 'right_shin', 'right_foot', [0, 0, 1])

8

In [98]:
startPose = env.flat_env.parts['torso'].get_pose()
print(startPose)

[ 0.          0.          1.17        0.          0.         -0.38268343
  0.92387953]


In [100]:
env.flat_env.parts['torso'].reset_pose([0, 0, 5], env.flat_env.parts['torso'].get_orientation())

In [104]:
for c in bvh_pos.columns:
    print(c)

Hips_Xposition
Hips_Yposition
Hips_Zposition
LowerBack_Xposition
LowerBack_Yposition
LowerBack_Zposition
Spine_Xposition
Spine_Yposition
Spine_Zposition
Spine1_Xposition
Spine1_Yposition
Spine1_Zposition
RightShoulder_Xposition
RightShoulder_Yposition
RightShoulder_Zposition
RightArm_Xposition
RightArm_Yposition
RightArm_Zposition
RightForeArm_Xposition
RightForeArm_Yposition
RightForeArm_Zposition
RightHand_Xposition
RightHand_Yposition
RightHand_Zposition
RThumb_Xposition
RThumb_Yposition
RThumb_Zposition
RThumb_Nub_Xposition
RThumb_Nub_Yposition
RThumb_Nub_Zposition
RightFingerBase_Xposition
RightFingerBase_Yposition
RightFingerBase_Zposition
RightHandIndex1_Xposition
RightHandIndex1_Yposition
RightHandIndex1_Zposition
RightHandIndex1_Nub_Xposition
RightHandIndex1_Nub_Yposition
RightHandIndex1_Nub_Zposition
LeftShoulder_Xposition
LeftShoulder_Yposition
LeftShoulder_Zposition
LeftArm_Xposition
LeftArm_Yposition
LeftArm_Zposition
LeftForeArm_Xposition
LeftForeArm_Yposition
LeftForeArm

In [108]:
bvh_pos['LowerBack_Xposition']

0 days 00:00:00               9.4455
0 days 00:00:00.008333300     9.4455
0 days 00:00:00.016666600     9.4469
0 days 00:00:00.024999900     9.4515
0 days 00:00:00.033333200     9.4570
                              ...   
0 days 00:00:03.991650700    10.2581
0 days 00:00:03.999984       10.2592
0 days 00:00:04.008317300    10.2622
0 days 00:00:04.016650600    10.2638
0 days 00:00:04.024983900    10.2703
Name: LowerBack_Xposition, Length: 484, dtype: float64

In [109]:
bvh_pos['Hips_Xposition']

0 days 00:00:00               9.4455
0 days 00:00:00.008333300     9.4455
0 days 00:00:00.016666600     9.4469
0 days 00:00:00.024999900     9.4515
0 days 00:00:00.033333200     9.4570
                              ...   
0 days 00:00:03.991650700    10.2581
0 days 00:00:03.999984       10.2592
0 days 00:00:04.008317300    10.2622
0 days 00:00:04.016650600    10.2638
0 days 00:00:04.024983900    10.2703
Name: Hips_Xposition, Length: 484, dtype: float64

In [43]:
pybullet.removeAllUserDebugItems()

In [102]:
pybullet.removeAllUserDebugItems()
i = 150
# drawLine2Joint(bvh_pos, 'Hips', 'Spine', i, [0, 0, 1])
# drawLine2Joint(bvh_pos, 'Spine', 'Spine1', i, [0, 1, 1])

drawLine2Joint(bvh_pos, 'LowerBack', 'LeftUpLeg', i, [1, 0, 0])
drawLine2Joint(bvh_pos, 'LeftUpLeg', 'LeftLeg', i, [1, 0, 0])
drawLine2Joint(bvh_pos, 'LeftLeg', 'LeftFoot', i, [1, 0, 0])

drawLine2Joint(bvh_pos, 'LowerBack', 'RightUpLeg', i, [0, 1, 0])
drawLine2Joint(bvh_pos, 'RightUpLeg', 'RightLeg', i, [0, 1, 0])
drawLine2Joint(bvh_pos, 'RightLeg', 'RightFoot', i, [0, 1, 0])

# drawLine2Joint(bvh_pos, 'Hips', 'RightShoulder', i, [1, 1, 0])
# drawLine2Joint(bvh_pos, 'RightShoulder', 'RightArm', i, [1, 1, 1])
drawLine2Joint(bvh_pos, 'RightArm', 'RightForeArm', i, [1, 1, 1])
drawLine2Joint(bvh_pos, 'RightForeArm', 'RightHand', i, [1, 1, 1])

# drawLine2Joint(bvh_pos, 'Hips', 'LeftShoulder', i, [0, 0, 0])
# drawLine2Joint(bvh_pos, 'LeftShoulder', 'LeftArm', i, [0, 0, 0])
drawLine2Joint(bvh_pos, 'LeftArm', 'LeftForeArm', i, [0, 0, 0])
drawLine2Joint(bvh_pos, 'LeftForeArm', 'LeftHand', i, [0, 0, 0])

# drawLine2Joint(bvh_pos, 'LeftLeg', 'LeftForeArm', i, [1, 1,1 ])

9

In [31]:
print(a)
print(b)
print(c)

0.27712812921102037
0.31176914536239786
0.29444863728670917


In [49]:
for i in range(100, 300):
    drawLine2Joint(bvh_pos, 'LowerBack', 'LeftUpLeg', i, [1, 0, 0])
    drawLine2Joint(bvh_pos, 'LeftUpLeg', 'LeftLeg', i, [1, 0, 0])
    drawLine2Joint(bvh_pos, 'LeftLeg', 'LeftFoot', i, [1, 0, 0])
    
    drawLine2Joint(bvh_pos, 'LowerBack', 'RightUpLeg', i, [0, 1, 0])
    drawLine2Joint(bvh_pos, 'RightUpLeg', 'RightLeg', i, [0, 1, 0])
    drawLine2Joint(bvh_pos, 'RightLeg', 'RightFoot', i, [0, 1, 0])

    drawLine2Joint(bvh_pos, 'Hips', 'RightShoulder', i, [1, 1, 1])
    drawLine2Joint(bvh_pos, 'RightShoulder', 'RightArm', i, [1, 1, 1])
    drawLine2Joint(bvh_pos, 'RightArm', 'RightForeArm', i, [1, 1, 1])
    drawLine2Joint(bvh_pos, 'RightForeArm', 'RightHand', i, [1, 1, 1])

    drawLine2Joint(bvh_pos, 'Hips', 'LeftShoulder', i, [0, 0, 0])
    drawLine2Joint(bvh_pos, 'LeftShoulder', 'LeftArm', i, [0, 0, 0])
    drawLine2Joint(bvh_pos, 'LeftArm', 'LeftForeArm', i, [0, 0, 0])
    drawLine2Joint(bvh_pos, 'LeftForeArm', 'LeftHand', i, [0, 0, 0])

    time.sleep(1.0/60)
    pybullet.removeAllUserDebugItems()

In [109]:
def setEnv(df):
    env.flat_env.jdict['right_knee'].set_state(df['rightKnee'], 0)

    env.flat_env.jdict['right_hip_x'].set_state(df['rightHipX'], 0)
    env.flat_env.jdict['right_hip_y'].set_state(df['rightHipY'], 0)
    env.flat_env.jdict['right_hip_z'].set_state(df['rightHipZ'], 0)
    
    env.flat_env.jdict['left_knee'].set_state(df['leftKnee'], 0)

    env.flat_env.jdict['left_hip_x'].set_state(df['leftHipX'], 0)
    env.flat_env.jdict['left_hip_y'].set_state(df['leftHipY'], 0)
    env.flat_env.jdict['left_hip_z'].set_state(df['leftHipZ'], 0)

setEnv(joint_df.iloc[0])

In [111]:
setEnv(joint_df.iloc[80])

In [114]:
for i in range(10):
    observation = env.resetNonFrame()
    idx = 0
    while(idx < len(joint_df)):
        # action = agent.compute_action(observation)
        # observation, reward, done, info = env.step(action)
        setEnv(joint_df.iloc[idx])
        idx += 1
        time.sleep(1.0/30)

In [115]:
env.close()